In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=d821391641d1dffd5c2ac3cf2dd2c28d249ab68201465abc7babdd38bb978251
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F2 '''
def F2(X):
    f = bn.F2()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 10202.350099442046 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_2_1000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_2_200Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 50, [5] * 50 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
        min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=930576, Mon Apr 13 18:39:36 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 2.743442874935384e+07 1.0e+00 2.36e+00  2e+00  2e+00 0:00.0
    2     30 2.297590051564116e+07 1.0e+00 2.27e+00  2e+00  2e+00 0:00.0
    3     45 1.767747415721293e+07 1.0e+00 2.21e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [27701781.24480721] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [44089684.27757464] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -7.456627686477450e+07 1.5e+00 3.85e-01  3e-01  4e-01 0:01.8
  200   3000 -8.004420157931748e+07 1.8e+00 2.39e-02  2e-02  2e-02 0:03.7
  300   4500 -8.006221928754234e+07 2.0e+00 1.73e-03  1e-03  2e-03 0:05.4
  400   6000 -8.006227858935544e+07 2.3e+00 1.14e-04  6e-05  1e-04 0:07.2
  500   7500 -8.006227898079285e+07 2.5e+00 9.74e-06  4e-06  9e-06 0:09.0
  600   9000 -8.006227898296410e+07 2.7e+00 5.12e-07  2e-07  4e-07 0:10.8
  700  10500 -8.006227898297095e+07 2.8e+00 4.71e-08  2e-08  4e-08 0:12.6
  800  12000 -8.006227898297095e+07 3.1e+00 2.02e-08  7e-09  2e-08 0:14.3
  900  13500 -8.006227898297098e+07 3.4e+00 1.53e-08  6e-09  1e-08 0:16.1
 1000  15000 -8.006227898297092e+07 3.5e+00 1.29e-08  5e-09  1e-08 0:17.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -8.006227898297092e+07 3.8e+00 9.14e-09  3e-09  8e-09 0:19.6
 1200  18000 -8.006227898297095e+07 4.0e+00 1.16e-08  4e-09  1e-08 0:21.4
 1300  19500 -8.006227898297098e+07 4.3e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [52123833.46010518] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=52123833.460105, sigma=2.37e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=52123833.460105, sigma=2.37e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

  100   1500 1.889032003999963e+06 1.6e+00 7.84e-01  7e-01  8e-01 0:02.1
  200   3000 8.570993133997321e+05 2.0e+00 2.22e-01  2e-01  2e-01 0:04.2
  300   4500 4.065910597756580e+05 3.3e+00 1.84e-01  1e-01  2e-01 0:06.3
  400   6000 2.144589752982557e+04 4.5e+00 1.91e-01  1e-01  3e-01 0:08.4
  500   7500 -1.843452468708903e+05 5.4e+00 9.50e-02  4e-02  1e-01 0:10.5
  600   9000 -3.084768980795071e+05 6.5e+00 9.41e-02  4e-02  1e-01 0:12.6
  700  10500 -3.796369229559079e+05 8.3e+00 7.60e-02  3e-02  1e-01 0:14.8
  800  12000 -4.290875411773920e+05 1.1e+01 5.40e-02  2e-02  1e-01 0:17.0
  900  13500 -4.429717864354327e+05 1.1e+01 2.36e-02  7e-03  5e-02 0:19.2
 1000  15000 -4.440857245854586e+05 1.1e+01 6.08e-03  2e-03  1e-02 0:21.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -4.442869319717735e+05 1.2e+01 5.14e-03  1e-03  1e-02 0:23.6
 1200  18000 -4.444909473122507e+05 2.0e+01 6.05e-03  1e-03  2e-02 0:25.8
 1300  19500 -4.458120482439101e+05 5.3e+01 

...

  100   1500 3.708998111284673e+06 1.7e+00 1.59e+00  1e+00  2e+00 0:02.1
  200   3000 1.039399843823522e+06 2.0e+00 3.65e-01  3e-01  4e-01 0:04.1
  300   4500 5.066113362368941e+05 2.7e+00 2.02e-01  1e-01  2e-01 0:06.2
  400   6000 2.426791491851360e+05 4.3e+00 1.44e-01  7e-02  2e-01 0:08.3
  500   7500 6.373077111321688e+04 5.9e+00 1.03e-01  5e-02  1e-01 0:10.4
  600   9000 -5.424991805650294e+03 6.9e+00 5.40e-02  2e-02  9e-02 0:12.4
  700  10500 -3.709228307676315e+04 8.1e+00 5.96e-02  2e-02  1e-01 0:14.5
  800  12000 -5.640508702251315e+04 9.3e+00 2.12e-02  7e-03  5e-02 0:16.6
  900  13500 -5.766347176560760e+04 9.8e+00 6.70e-03  2e-03  1e-02 0:18.6
 1000  15000 -5.781630006082356e+04 1.1e+01 4.01e-03  1e-03  9e-03 0:20.7
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -5.824239512120187e+04 2.9e+01 1.06e-02  2e-03  7e-02 0:22.7
 1200  18000 -5.945499715143442e+04 5.5e+01 7.69e-03  2e-03  8e-02 0:24.8
 1300  19500 -5.958299351151288e+04 5.5e+01 2

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [74894040.78064375] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [53848600.96899728] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -6.978244362055525e+07 1.5e+00 6.38e-01  5e-01  7e-01 0:02.0
  200   3000 -7.999947422915077e+07 1.7e+00 4.35e-02  3e-02  4e-02 0:03.9
  300   4500 -8.006190769438905e+07 2.0e+00 3.52e-03  2e-03  4e-03 0:05.9
  400   6000 -8.006227723876378e+07 2.2e+00 2.47e-04  1e-04  2e-04 0:07.8
  500   7500 -8.006227897283152e+07 2.5e+00 1.93e-05  9e-06  2e-05 0:09.8
  600   9000 -8.006227898291269e+07 2.7e+00 1.74e-06  7e-07  2e-06 0:11.7
  700  10500 -8.006227898297060e+07 2.9e+00 1.40e-07  5e-08  1e-07 0:13.7
  800  12000 -8.006227898297089e+07 3.0e+00 2.73e-08  1e-08  2e-08 0:15.6
  900  13500 -8.006227898297095e+07 3.4e+00 2.95e-08  1e-08  3e-08 0:17.6
 1000  15000 -8.006227898297098e+07 3.6e+00 1.77e-08  7e-09  2e-08 0:19.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -8.006227898297098e+07 4.0e+00 2.00e-08  7e-09  2e-08 0:21.4
 1200  18000 -8.006227898297095e+07 4.4e+00 1.55e-08  6e-09  1e-08 0:23.4
 1300  19500 -8.006227898297095e+07 4.8e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [52123833.46010518] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=52123833.460105, sigma=2.36e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=52123833.460105, sigma=2.36e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

  100   1500 3.151348464704514e+06 1.7e+00 1.24e+00  1e+00  1e+00 0:02.1
  200   3000 1.068795998287149e+06 2.1e+00 4.13e-01  3e-01  5e-01 0:04.1
  300   4500 5.133480012704805e+05 2.6e+00 1.62e-01  1e-01  2e-01 0:06.2
  400   6000 2.041517697220594e+05 4.3e+00 1.28e-01  7e-02  2e-01 0:08.2
  500   7500 -2.319267768046260e+05 6.7e+00 1.65e-01  7e-02  2e-01 0:10.3
  600   9000 -4.276252111553550e+05 7.3e+00 6.59e-02  3e-02  9e-02 0:12.3
  700  10500 -4.431173960448951e+05 7.5e+00 3.64e-02  1e-02  5e-02 0:14.4
  800  12000 -4.461703497095481e+05 7.8e+00 6.08e-03  2e-03  9e-03 0:16.4
  900  13500 -4.463389021175429e+05 8.3e+00 3.39e-03  9e-04  5e-03 0:18.5
 1000  15000 -4.464651662229970e+05 1.4e+01 3.74e-03  1e-03  1e-02 0:20.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -4.471226251021028e+05 3.9e+01 1.36e-02  3e-03  1e-01 0:22.6
 1200  18000 -4.474906022122577e+05 4.2e+01 2.99e-03  7e-04  3e-02 0:24.6
 1300  19500 -4.475250710688084e+05 4.2e+01 

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F2_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F2(X_Values.iloc[i,:50])
    SVM_Fun [i] = F2(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F2(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([10579934.39325758]), count=array([1])) 0.1306893353082328
SVM
ModeResult(mode=array([23327485.52068637]), count=array([1])) 26239024.734131247
ELN
ModeResult(mode=array([703928.39880273]), count=array([1])) 84370.25182450759
